# GCP Resources Setup

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

import kfp

from kfp.v2 import compiler, dsl
from kfp.v2.dsl import pipeline, component, Artifact, Dataset, Input, Metrics, Model, Output, InputPath, OutputPath, ClassificationMetrics
from typing import NamedTuple

from google.cloud import aiplatform

# We'll use this namespace for metadata querying
from google.cloud import aiplatform_v1

from google.cloud.aiplatform import pipeline_jobs
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google.cloud import bigquery

import os
PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
    
    
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin
REGION="us-central1"

from datetime import datetime

TIMESTAMP =datetime.now().strftime("%Y%m%d%H%M%S")


Project ID:  vertex-testing-327520
env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


## BigQuery - Database (Raw stage)

Necesitamos crear dos datasets en BigQuery: uno que oficiará de histórico, y otro que representará el momento actual (no va a tener variable target). Para eso usaremos el gcloud cli, junto con algunas variables de Python. 

**Importante**: Este código tiene que correrse sólo una vez, es para crear las bases necesarias que se usarán a lo largo del pipeline.

We need to create two datasets in BigQuery: one will be our historical data, and the other one will simulate the current time (the target variable will be missing). We'll use the gcloud cli, along with some Python variables.

**Important**: This code needs to run only once to set up the necessary datasets to be used through the entire pipeline.

In [20]:
BQ_DATASET_HISTORIC_NAME = 'chicago_taxi_historic_test'
BQ_DATASET_CURRENT_NAME = 'chicago_taxi_current_test'

BQ_HISTORIC_RAW = 'raw'
BQ_CURRENT_RAW = 'raw'

BQ_LOCATION = 'US'

In [3]:
# !bq --location=US mk -d \
# $PROJECT_ID:$BQ_DATASET_HISTORIC_NAME

Dataset 'vertex-testing-327520:chicago_taxi_historic_test' successfully created.


In [4]:
# !bq --location=US mk -d \
# $PROJECT_ID:$BQ_DATASET_CURRENT_NAME

Dataset 'vertex-testing-327520:chicago_taxi_current_test' successfully created.


Se preparan dos funciones para automatizar y parametrizar la búsqueda de variables de acuerdo a tiempos y volúmenes deseados.

We create two functions in order to automate and parametrize variables according to times and size needed.

In [21]:
import datetime as dt

def get_year_and_month():
    previous_month = (dt.date.today().replace(day=1) - dt.timedelta(days=33)).month
    year = dt.date.today().year
    
    if previous_month == 12:
        year = year-1
    else:
        year
    return year, previous_month

In [22]:
def get_year_and_month_hist(current_year, current_month):
    month_hist = current_month - 1
    if month_hist == 0:
        year_hist = current_year -1
        month_hist = 12
    else:
        year_hist = current_year
    
    return year_hist, month_hist

In [23]:
import datetime as dt
SAMPLE_SIZE = 100000
YEAR, MONTH = get_year_and_month()

print('current year: ', YEAR)
print('current month: ', MONTH)
print('sample size: ', SAMPLE_SIZE)

current year:  2021
current month:  12
sample size:  100000


In [24]:
HIST_YEAR, HIST_MONTH = get_year_and_month_hist(YEAR, MONTH)
print('past year: ', HIST_YEAR)
print('past month: ', HIST_MONTH)

past year:  2021
past month:  11


### Dataset actual - Current dataset

Usamos una query de SQL para poblar la tabla.

We use a SQL query to load the table.

In [25]:
current_sql_script = '''
CREATE OR REPLACE TABLE `@PROJECT_ID.@DATASET.@TABLE` 
AS (
    WITH
      taxitrips AS (
      SELECT
        trip_start_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        fare
      FROM
        `bigquery-public-data.chicago_taxi_trips.taxi_trips`
      WHERE 1=1 
      AND pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = @YEAR
      AND EXTRACT(MONTH FROM trip_start_timestamp) = @MONTH
    )

    SELECT
      trip_start_timestamp,
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude), 
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      CONCAT(
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickup_longitude,
              pickup_latitude), 0.1)), 
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropoff_longitude,
              dropoff_latitude), 0.1))
      ) AS loc_cross,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
      IF(ABS(MOD(FARM_FINGERPRINT(STRING(trip_start_timestamp)), 10)) < 9, 'UNASSIGNED', 'TEST') AS data_split
    FROM
      taxitrips
    LIMIT @LIMIT
)
'''

In [26]:
current_sql_script = current_sql_script.replace(
    '@PROJECT_ID', PROJECT_ID).replace(
    '@DATASET', BQ_DATASET_CURRENT_NAME).replace(
    '@TABLE', BQ_CURRENT_RAW).replace(
    '@YEAR', str(YEAR)).replace(
    '@LIMIT', str(SAMPLE_SIZE)).replace(
    '@MONTH', str(MONTH))

In [27]:
bq_client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)
job = bq_client.query(current_sql_script)
_ = job.result()

### Dataset historico - Historic dataset

In [28]:
historic_sql_script = '''
CREATE OR REPLACE TABLE `@PROJECT_ID.@DATASET.@TABLE` 
AS (
    WITH
      taxitrips AS (
      SELECT
        trip_start_timestamp,
        trip_seconds,
        trip_miles,
        payment_type,
        pickup_longitude,
        pickup_latitude,
        dropoff_longitude,
        dropoff_latitude,
        tips,
        fare
      FROM
        `bigquery-public-data.chicago_taxi_trips.taxi_trips`
      WHERE 1=1 
      AND pickup_longitude IS NOT NULL
      AND pickup_latitude IS NOT NULL
      AND dropoff_longitude IS NOT NULL
      AND dropoff_latitude IS NOT NULL
      AND trip_miles > 0
      AND trip_seconds > 0
      AND fare > 0
      AND EXTRACT(YEAR FROM trip_start_timestamp) = @YEAR
      AND EXTRACT(MONTH FROM trip_start_timestamp) = @MONTH
    )

    SELECT
      trip_start_timestamp,
      EXTRACT(MONTH from trip_start_timestamp) as trip_month,
      EXTRACT(DAY from trip_start_timestamp) as trip_day,
      EXTRACT(DAYOFWEEK from trip_start_timestamp) as trip_day_of_week,
      EXTRACT(HOUR from trip_start_timestamp) as trip_hour,
      trip_seconds,
      trip_miles,
      payment_type,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(pickup_longitude, pickup_latitude), 0.1)
      ) AS pickup_grid,
      ST_AsText(
          ST_SnapToGrid(ST_GeogPoint(dropoff_longitude, dropoff_latitude), 0.1)
      ) AS dropoff_grid,
      ST_Distance(
          ST_GeogPoint(pickup_longitude, pickup_latitude), 
          ST_GeogPoint(dropoff_longitude, dropoff_latitude)
      ) AS euclidean,
      CONCAT(
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(pickup_longitude,
              pickup_latitude), 0.1)), 
          ST_AsText(ST_SnapToGrid(ST_GeogPoint(dropoff_longitude,
              dropoff_latitude), 0.1))
      ) AS loc_cross,
      IF((tips/fare >= 0.2), 1, 0) AS tip_bin,
      IF(ABS(MOD(FARM_FINGERPRINT(STRING(trip_start_timestamp)), 10)) < 9, 'UNASSIGNED', 'TEST') AS data_split
    FROM
      taxitrips
    LIMIT @LIMIT
)
'''

In [29]:
historic_sql_script = historic_sql_script.replace(
    '@PROJECT_ID', PROJECT_ID).replace(
    '@DATASET', BQ_DATASET_HISTORIC_NAME).replace(
    '@TABLE', BQ_HISTORIC_RAW).replace(
    '@YEAR', str(HIST_YEAR)).replace(
    '@LIMIT', str(SAMPLE_SIZE)).replace(
    '@MONTH', str(HIST_MONTH))

In [30]:
bq_client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)
job = bq_client.query(historic_sql_script)
_ = job.result()

## Cloud Storage - Artifacts

Crearemos buckets en GCS para almacenar distintos tipos de objetos que va produciendo el pipeline a lo largo del camino. En *stage* se guardarán algunos más relevantes, como respaldos de las particiones de train, validación y test para poder acceder más rapidamente y fácilmente, mientras que en *pipelines* habrá mayormente logs y resultados de ejecuciones.

We'll create GCS buckets to store different types of objects that the pipeline produces through its execution. In *stage* there will be some relevant files, such as easily accesible backups of train, validation and test data in order to perform quick reviews if needed, whereas in *pipeline* there will mostly be execution outputs and logs.

In [2]:
STAGE_DATA_BUCKET = f'{PROJECT_ID}-chicago_taxi_stage'
PIPELINE_DATA_BUCKET = f'{PROJECT_ID}-chicago_taxi_pipelines'

print('Stage bucket: ', STAGE_DATA_BUCKET)
print('Pipeline bucket: ', PIPELINE_DATA_BUCKET)


Stage bucket:  vertex-testing-327520-chicago_taxi_stage
Pipeline bucket:  vertex-testing-327520-chicago_taxi_pipelines


In [3]:
from google.cloud import storage


def create_bucket_class_location(bucket_name, location):
    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    new_bucket = storage_client.create_bucket(bucket, location=location)

    print(
        "Created bucket {} in {} ".format(
            new_bucket.name, new_bucket.location
        )
    )
    return new_bucket

In [4]:
bucket_stage = create_bucket_class_location(STAGE_DATA_BUCKET, REGION)

Created bucket vertex-testing-327520-chicago_taxi_stage in US-CENTRAL1 


In [5]:
bucket_pipeline = create_bucket_class_location(PIPELINE_DATA_BUCKET, REGION)

Created bucket vertex-testing-327520-chicago_taxi_pipelines in US-CENTRAL1 
